In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-143981
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-143981


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

#choose a name for the cluster
cpu_clusterName = 'compute-cluster'

#verify that the cluster does not exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_clusterName)
    print('found existing cluster, so use this')
except ComputeTargetException:
    print('no cluster was found, create a new cluster')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_clusterName, compute_config)

compute_target.wait_for_completion(show_output=True)

#get details of the status of the cluster

#print(compute_target.get_status().serialize())


found existing cluster, so use this
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [29]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig, Experiment, Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
 

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100),
        '--max_iter': choice(50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

#env = Environment.get(workspace=ws, name='AzureML')
# Create a SKLearn estimator for use with train.py
#est = ScriptRunConfig(source_directory = "./",
#            compute_target=compute_target,
#            script="train.py",
#            environment=env)

est = SKLearn(source_directory = "./",
              entry_script = 'train.py',
              compute_target=cpu_clusterName,
              vm_size = "Standard_D2_V2" )
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     estimator=est,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

In [31]:

# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

 

RunId: HD_131dffa5-32a7-401e-b7a6-3bdf2c3fa7d9
Web View: https://ml.azure.com/runs/HD_131dffa5-32a7-401e-b7a6-3bdf2c3fa7d9?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-143981/workspaces/quick-starts-ws-143981&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-04T23:23:17.270832][API][INFO]Experiment created<END>\n""<START>[2021-05-04T23:23:17.705064][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-05-04T23:23:17.881483][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [28]:
import joblib
# Get your best run and 

 
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print(parameters)
print(best_run_metric)


#save the model from that run.
hyperdrive_model = best_run.register_model(model_name='hyperdrive_model',model_path='outputs/model.joblib')


None


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
urls= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(urls)
### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###